Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>


В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
res = requests.get('https://habr.com/ru/')
# res
res.status_code

200

In [3]:
soup = BeautifulSoup(res.text)
soup

<!DOCTYPE html>
<html data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D" lang="ru">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,viewport-fit=cover,maximum-scale=1,user-scalable=0" name="viewport"/>
<meta content="unsafe-url" name="referrer"/>
<title>Все публикации подряд / Хабр</title>
<style>
    /* cyrillic-ext */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format('woff2');
      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;
    }

    /* cyrillic */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format('woff2');
      unicode-range: U+0400-045F, U+0490-0

In [4]:
# posts = soup.find_all('div', class_='tm-article-snippet tm-article-snippet')
# print(posts[0])
posts = soup.find_all('article', class_='tm-articles-list__item')

In [47]:
for el in posts:
    date = el.find('time').get('title')
    print(date)
    title = el.find('a', 'tm-article-snippet__title-link').text
    print(title)
  
    link = el.find('a', 'tm-article-snippet__title-link').get('href')
    print(f'https://habr.com{link}')    
    print()

2023-02-03, 17:22
Компьютерное зрение: обучение Dense Neural Network
https://habr.com/ru/company/otus/blog/714754/

2023-02-03, 17:12
Причины минусов в карму и история голосов v2.01
https://habr.com/ru/post/714734/

2023-02-03, 17:10
Kubernetes в Docker (KinD): балансировка нагрузки Kubernetes в Docker
https://habr.com/ru/company/otus/blog/714748/

2023-02-03, 16:53
Учёным впервые удалось изучить структуру протона при помощи нейтрино
https://habr.com/ru/post/714738/

2023-02-03, 16:52
Семь красных линий
https://habr.com/ru/post/714736/

2023-02-03, 16:49
Приглашаем на Ozon Tech Community Mobile Meetup
https://habr.com/ru/company/ozontech/blog/714716/

2023-02-03, 16:38
Как запатентовать идею: 3 инструкции по заполнению заявки на патент
https://habr.com/ru/post/714728/

2023-02-03, 16:33
Список ненужных задач в Планировщике заданий Windows
https://habr.com/ru/post/714694/

2023-02-03, 16:03
Что ждет ручное тестирование в 2023 году
https://habr.com/ru/company/otus/blog/714188/

2023-02-0

In [5]:
def get_habr_blog():
    habr_blog = pd.DataFrame()
    
    url = 'https://habr.com/ru/'
    req = requests.get(url).text
    time.sleep(0.3)
    soup = BeautifulSoup(req) 
    posts = soup.find_all('article', class_='tm-article-list__item')

    habr_blog = pd.DataFrame()

    for el in posts:
        title = el.find('a', 'tm-article-snippet__title-link').text
        link = el.find('a', 'tm-article-snippet__title-link').get('href')
        date = el.find('time').get('title')
        category = el.find('a', 'tm-article-snippet__hubs-item-link router-link-exact-active router-link-active').text
        row = {'date': date, 'title': title, 'link': link}
        netology_blog = pd.concat([habr_blog, pd.DataFrame([row])]) 
    return habr_blog.reset_index(drop=True)

res = get_habr_blog()
res

""


In [6]:
def get_netology_posts(query):
    netology_blog = pd.DataFrame()
    for q in query:
        URL = 'https://netology.ru/blog/'
        params = {
            's': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='status-publish')
        for article in articles:
            title = article.find('h2', class_='entry-title').text
            link = article.find('h2', class_='entry-title').find('a').get('href')
            date = article.find('span', class_='posted-on').text.strip()
            if article.find('div', class_='entry-cats'):
                category = article.find('div', class_='entry-cats').text
            else:
                category = 'Новость'
            row = {'date': date, 'title': title, 'link': link, 'category':category}
            netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 
    return netology_blog.reset_index(drop=True)

res = get_netology_posts(['python', 'data science'])
res

,date,title,link,category
0,12 октября 2021,Python занял первое место в рейтинге языков пр...,https://netology.ru/blog/news/12-10-2021-python,Новость
1,31 июля 2019,С чего начать изучение Python: 3 первых шага,https://netology.ru/blog/07-2019-python-3-perv...,Современные профессии
2,15 марта 2019,С чего начать изучение Python: книги для начин...,https://netology.ru/blog/03-2019-python-knigi-...,Книжная полка
3,16 января 2019,"Как перейти в Machine learning, если знаешь Py...",https://netology.ru/blog/2019-01-machine-learn...,Карьера
4,15 января 2019,Создание модели распознавания лиц с использова...,https://netology.ru/blog/2019-01-modeli-raspoz...,Руководство
5,30 декабря 2016,Как парсить данные с HTML-страниц на Python,https://netology.ru/blog/parsing-python,Руководство
6,16 января 2023,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/01-2023-data-science-...,Книжная полка
7,15 декабря 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/12-2022-data-science-...,Книжная полка
8,15 ноября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/11-2022-data-science-...,Книжная полка
9,12 октября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/10-2022-data-science-...,Книжная полка


In [7]:
def get_netology_blog_posts():
    url = 'https://netology.ru/blog/'
    req = requests.get(url).text
    soup = BeautifulSoup(req)

    news = soup.find_all('div', class_='posts__item')

    netology_blog = pd.DataFrame()

    for el in news:
        title = el.find('a', 'posts__link').text
        link = el.find('a', 'posts__link').get('href')
        date = el.find('div', 'posts__date').text
        category = el.find('a', 'tags__item').text
        row = {'date': date, 'title': title, 'link': link, 'category':category}
        netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 
    return netology_blog.reset_index(drop=True)

get_netology_blog_posts()

,date,title,link,category
0,3 февраля 2023,Внешние факторы и собственные прогнозы: почему...,https://netology.ru/blog/02-2022-mistakes,Руководство
1,3 февраля 2023,Как построить личную образовательную траектори...,https://netology.ru/blog/02-2023-educational-path,Научиться учиться
2,1 февраля 2023,Ставит чёткие ТЗ или умеет кодить: спросили у ...,https://netology.ru/blog/02-2023-ideal-product...,Личный опыт
3,31 января 2023,Продвижение интернет-магазина: какие сервисы Я...,https://netology.ru/blog/01-2023-online-store-...,Разобраться
4,30 января 2023,Кто такой саунд-дизайнер и как им стать,https://netology.ru/blog/01-2023-sound-designer,Современные профессии
5,27 января 2023,12 книг о целеполагании и продуктивности,https://netology.ru/blog/01-2023-books-about-g...,Книжная полка
6,26 января 2023,Релокация: как платить налоги в разных странах,https://netology.ru/blog/01-2023-pay-taxes-abroad,Разобраться
7,25 января 2023,Личный опыт: как начинающим владельцам продукт...,https://netology.ru/blog/01-2023-product-owner...,Продуктивность
8,24 января 2023,Кто такой менеджер бизнес-процессов и как он п...,https://netology.ru/blog/01-2023-manager-b-p,Современные профессии
9,23 января 2023,8 ошибок при запуске образовательного онлайн-п...,https://netology.ru/blog/01-2023-edmarket-8-mi...,Разобраться
